# Pré-processamento

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("denkuznetz/food-delivery-time-prediction")

print("Path to dataset files:", path)

In [ ]:
import os

# Listar os arquivos no diretório
files = os.listdir(path)
print("Arquivos no diretório:", files)

In [ ]:
import pandas as pd

csv_file_path = os.path.join(path, 'Food_Delivery_Times.csv')

df = pd.read_csv(csv_file_path)
df.head()

In [ ]:
from sklearn.impute import SimpleImputer

# Remover a coluna Order_ID
df.drop(columns=['Order_ID'], inplace=True)

# Substituir valores nulos em "Courier_Experience_yrs" pela mediana
imputer_num = SimpleImputer(strategy="median")
df["Courier_Experience_yrs"] = imputer_num.fit_transform(df[["Courier_Experience_yrs"]])

# Substituir valores nulos em variáveis categóricas pela moda
categorical_cols = ["Weather", "Traffic_Level", "Time_of_Day"]
imputer_cat = SimpleImputer(strategy="most_frequent")
df[categorical_cols] = imputer_cat.fit_transform(df[categorical_cols])

# Aplicar One-Hot Encoding nas variáveis categóricas
df = pd.get_dummies(df, columns=["Weather", "Traffic_Level", "Time_of_Day", "Vehicle_Type"], drop_first=True)

# Exibir as primeiras linhas do dataframe processado
df.head()


In [ ]:
for col in df.columns:
    if df[col].dtype == 'object':
        # If column is of object type, try converting to numeric
        try:
            df[col] = pd.to_numeric(df[col])
        except ValueError:
            # If conversion fails, handle the non-numeric values (e.g., replace with NaN or a specific value)
            # Here, we replace non-numeric values with NaN and then fill them with the column's median
            df[col] = pd.to_numeric(df[col], errors='coerce')  # Convert non-numeric to NaN
            df[col] = df[col].fillna(df[col].median())  # Fill NaN with median

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=['Delivery_Time_min']).values
y = df['Delivery_Time_min'].values
# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Selecionando as três primeiras colunas (dados contínuos)
continuous_indices = [0, 1, 2]

# Aplicando MinMaxScaler
scaler = MinMaxScaler()
X_train[:, continuous_indices] = scaler.fit_transform(X_train[:, continuous_indices])
X_test[:, continuous_indices] = scaler.transform(X_test[:, continuous_indices])

In [ ]:
# Normalizando o alvo
y_min = y_train.min()
y_max = y_train.max()

y_train = (y_train - y_min) / (y_max - y_min)
y_test = (y_test - y_min) / (y_max - y_min)


In [ ]:
import numpy as np
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

# Código da semana passado

In [ ]:
import torch

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

from torch.utils.data import Dataset, DataLoader

class RegressionDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Criar datasets para treino e teste
train_dataset = RegressionDataset(X_train_tensor, y_train_tensor)
test_dataset = RegressionDataset(X_test_tensor, y_test_tensor)

batch_size = 32  # Escolha um tamanho de batch
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

import torch.nn as nn

class RegressionModel(nn.Module):
    def __init__(self, input_size):
        super(RegressionModel, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 64),  # Primeira camada
            nn.ReLU(),                 # Função de ativação
            nn.Linear(64, 32),         # Segunda camada
            nn.ReLU(),
            nn.Linear(32, 1)           # Camada de saída
        )

    def forward(self, x):
        return self.model(x)

# Inicializar o modelo
input_size = X_train_tensor.shape[1]  # Número de recursos (features)
model = RegressionModel(input_size)

import torch.optim as optim

# Função de custo
criterion = nn.MSELoss()

# Otimizador
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Taxa de aprendizado

# Configurações do treinamento
epochs = 100  # Número de épocas
model.train()  # Colocar o modelo em modo de treinamento

for epoch in range(epochs):
    epoch_loss = 0.0
    for X_batch, y_batch in train_loader:
        # Zerar os gradientes
        optimizer.zero_grad()

        # Forward pass
        predictions = model(X_batch)

        # Calcular a perda
        loss = criterion(predictions, y_batch)

        # Backward pass
        loss.backward()

        # Atualizar os pesos
        optimizer.step()

        # Acumular a perda
        epoch_loss += loss.item()

    print(f"Época {epoch+1}/{epochs}, Perda: {epoch_loss/len(train_loader):.4f}")

model.eval()  # Colocar o modelo em modo de avaliação
test_loss = 0.0

with torch.no_grad():  # Desativar o cálculo do gradiente
    for X_batch, y_batch in test_loader:
        predictions = model(X_batch)
        loss = criterion(predictions, y_batch)
        test_loss += loss.item()

print(f"Perda no conjunto de teste: {test_loss/len(test_loader):.4f}")




# Otimização

https://optuna.org/#code_examples

https://www.geeksforgeeks.org/hyperparameter-tuning-with-optuna-in-pytorch/

In [ ]:
!pip install optuna

In [ ]:
import optuna

# Tarefa 1: Recriar o modelo

# Tarefa 2: Criar função objetivo para o Optuna

# Tafera 3: Rodar a otimização com o optuna

# Tarefa: Hiperparâmetros com LLMs

1. Descrição do Problema: Forneça um prompt inicial para a LLM com uma descrição clara do problema, incluindo:

O modelo que você está treinando.
A estrutura dos dados (número de features, etc.).
Os hiperparâmetros que você deseja otimizar.
O orçamento de busca (número de configurações a serem testadas).

2. Sugestão de Hiperparâmetros: A LLM gera uma configuração de hiperparâmetros no formato que você especificar (ex.: JSON).

3. Treinamento e Avaliação: Use os hiperparâmetros sugeridos pela LLM para treinar o modelo e calcule a métrica de validação (ex.: perda ou acurácia).

4. Feedback: Envie o resultado da avaliação de volta para a LLM, junto com o histórico de configurações testadas, para que ela sugira a próxima configuração.

5. Iteração: Repita o processo até que o orçamento de busca seja esgotado.

In [ ]:
hyperparameters = {
  "learning_rate": ,
  "hidden1": ,
  "hidden2": ,
  "batch_size":
}

In [ ]:
# Configuração do modelo com os hiperparâmetros sugeridos
model = RegressionModel(input_size, hyperparameters["hidden1"], hyperparameters["hidden2"])
optimizer = optim.Adam(model.parameters(), lr=hyperparameters["learning_rate"])
batch_size = hyperparameters["batch_size"]

# Atualize o DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Treinamento do modelo
model.train()
for epoch in range(10):  # Ajuste o número de épocas conforme necessário
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        predictions = model(X_batch)
        loss = criterion(predictions, y_batch)
        loss.backward()
        optimizer.step()

# Avaliação no conjunto de validação
model.eval()
val_loss = 0.0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        predictions = model(X_batch)
        val_loss += criterion(predictions, y_batch).item()

val_loss /= len(test_loader)
print(f"Validação Loss: {val_loss}")
